In [1]:
import pandas as pd
import apriori_functions
import define_
from mlxtend.frequent_patterns import apriori
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import association_rules

In [2]:
dataSet = pd.read_csv(define_.DATA_FILE_PATH)
dataSet.head()

,No.,Time,Source,Destination,Protocol,Length,Info,Src_port,Dst_port,Class
0,1,0.000000,11.0.0.192,10.0.0.13,TCP,74,45414 > 80 [SYN] Seq=0 Win=28200 Len=0 MSS=1...,45414.0,80.0,A2
1,2,0.001978,11.0.0.85,10.0.0.13,TCP,74,40847 > 80 [SYN] Seq=0 Win=28200 Len=0 MSS=1...,40847.0,80.0,A2
2,3,0.002053,11.0.0.192,10.0.0.13,TCP,66,45414 > 80 [ACK] Seq=1 Ack=1 Win=28224 Len=0...,45414.0,80.0,A2
3,4,0.002106,11.0.0.192,10.0.0.13,HTTP,139,GET / HTTP/1.1,45414.0,80.0,A2
4,5,0.002186,11.0.0.85,10.0.0.13,TCP,66,40847 > 80 [ACK] Seq=1 Ack=1 Win=28224 Len=0...,40847.0,80.0,A2


In [3]:
dataSet = apriori_functions.preProcessing(dataSet)
dataSet

,Protocol,Length,Dst_port,Class,Dst_ip_range,Direction
0,p-TCP,l-1,d-80.0,c-A2,r-PRIVATE,x-OUT
1,p-TCP,l-1,d-80.0,c-A2,r-PRIVATE,x-OUT
2,p-TCP,l-1,d-80.0,c-A2,r-PRIVATE,x-OUT
3,p-HTTP,l-1,d-80.0,c-A2,r-PRIVATE,x-OUT
4,p-TCP,l-1,d-80.0,c-A2,r-PRIVATE,x-OUT
...,...,...,...,...,...,...
1048569,p-TCP,l-1,d-80.0,c-A1,r-PRIVATE,x-OUT
1048570,p-TCP,l-1,d-80.0,c-A1,r-PRIVATE,x-OUT
1048571,p-HTTP,l-1,d-80.0,c-A1,r-PRIVATE,x-OUT
1048572,p-TCP,l-1,d-80.0,c-A1,r-PRIVATE,x-OUT


In [13]:
dataSet.columns

Index(['Protocol', 'Length', 'Dst_port', 'Class', 'Dst_ip_range', 'Direction'], dtype='object')

In [4]:
te = TransactionEncoder()
te_ary = te.fit(dataSet.to_numpy()).transform(dataSet.to_numpy(), sparse=True)
transformed_df = pd.DataFrame.sparse.from_spmatrix(te_ary, columns=te.columns_)
transformed_df

,c-A1,c-A2,c-A3,d-22.0,d-32768.0,d-32770.0,d-32772.0,d-32774.0,d-32776.0,d-32778.0,...,p-DNS,p-HTTP,p-SSH,p-SSHv2,p-TCP,r-NON,r-PRIVATE,r-PUBLIC,x-NON,x-OUT
0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,True,0,0,True
1,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,True,0,0,True
2,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,True,0,0,True
3,0,1,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,True,0,0,True
4,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,True,0,0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048569,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,True,0,0,True
1048570,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,True,0,0,True
1048571,1,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,True,0,0,True
1048572,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,True,0,0,True


In [5]:
freqItemsets = apriori(transformed_df, min_support=0.1, use_colnames=True)
freqItemsets.to_csv(define_.FREQ_ITEMS_FILE_PATH)

In [6]:
freq = pd.read_csv(define_.FREQ_ITEMS_FILE_PATH)
df = apriori_functions.freqItemToDF(freq.to_numpy())
df.to_csv(define_.FREQ_ITEM_FORMATTED_FILE_PATH)

TypeError: 'int' object is not iterable

In [6]:
rules = association_rules(freqItemsets, metric="confidence", min_threshold=0.1)
rules = rules[(rules['antecedents'] == {'c-A1'}) |
                (rules['antecedents'] == {'c-A2'}) |
                (rules['antecedents'] == {'c-A3'}) ]
type(rules)
rules['antecedents'] = rules.apply(lambda row: apriori_functions.convertToStringList(str(list(row['antecedents']))), axis=1)
rules['consequents'] = rules.apply(lambda row: apriori_functions.convertToStringList(str(list(row['consequents']))), axis=1)
rules.reset_index(inplace=True)
rules.to_csv(define_.FINAL_RULES_FILE_PATH, index=False)
rules

,index,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,0,[c-A1],[l-0],0.255297,0.188013,0.187633,0.734959,3.909076,0.139634,3.063624
1,2,[c-A1],[p-TCP],0.255297,0.829059,0.245762,0.962652,1.161138,0.034106,4.576964
2,4,[c-A1],[r-PRIVATE],0.255297,0.999937,0.255286,0.999955,1.000018,0.000005,1.404135
3,6,[c-A1],[x-OUT],0.255297,0.999960,0.255291,0.999978,1.000018,0.000005,1.787080
4,8,[c-A2],[d-80.0],0.191868,0.747175,0.191626,0.998737,1.336685,0.048267,200.257518
...,...,...,...,...,...,...,...,...,...,...
72,1075,[c-A3],"[r-PRIVATE, l-1, x-OUT, d-80.0]",0.552835,0.746699,0.487641,0.882074,1.181299,0.074840,2.147976
73,1106,[c-A3],"[p-TCP, r-PRIVATE, x-OUT, d-80.0]",0.552835,0.641409,0.418851,0.757642,1.181215,0.064258,1.479592
74,1135,[c-A3],"[r-PRIVATE, p-TCP, l-1, x-OUT]",0.552835,0.641078,0.418748,0.757456,1.181534,0.064337,1.479820
75,1258,[c-A2],"[x-OUT, d-80.0, p-TCP, l-1, r-PRIVATE]",0.191868,0.640933,0.164116,0.855359,1.334553,0.041142,2.482471


In [2]:
rules = pd.read_csv(define_.FINAL_RULES_FILE_PATH)

In [3]:
# r, c = final_Rules.shape
featureList = [define_.P_TCP, define_.P_HTTP,define_.P_SSH,define_.P_DNS,define_.P_ARP, define_.P_SSHV2,define_.L_0, define_.L_1, define_.L_2, define_.D_80, define_.D_42972, define_.D_34230, define_.D_50822, define_.D_53, define_.D_22, define_.D_56040, define_.D_161, define_.D_443,
                                      define_.R_PUBLIC, define_.R_PRIVATE, define_.R_NON, define_.X_IN, define_.X_OUT, define_.X_NON, define_.C_A1, define_.C_A2, define_.C_A3]

oneHotEncodedDF = pd.DataFrame(apriori_functions.oneHot(rules,featureList), columns=featureList)
oneHotEncodedDF.to_csv(define_.TRAIN_ENCODED)

In [4]:
oneHotEncodedDF



,p-TCP,p-HTTP,p-SSH,p-DNS,p-ARP,p-SSHv2,l-0.0,l-1.0,l-2.0,d-80,...,d-443,r-PUBLIC,r-PRIVATE,r-NON,x-IN,x-OUT,x-NON,c-A1,c-A2,c-A3
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
73,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
74,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
75,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
